In [ ]:
import earthaccess
lon_min = -10
lat_min = 30
lon_max = 35
lat_max = 55

auth = earthaccess.login()
results = earthaccess.search_data(
    short_name="OCO2_L2_Lite_FP",
    temporal=("2024-01-01", "2025-01-02"),
    bounding_box=(-10, 30, 35, 55)
)

In [3]:
print(len(results))
print(results[0].data_links()[0])

18
https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MCD12Q1.061/MCD12Q1.A2024001.h20v04.061.2025205181956/MCD12Q1.A2024001.h20v04.061.2025205181956.hdf


In [3]:
files = earthaccess.download(results[:5], '../data/raw')

QUEUEING TASKS | :   0%|          | 0/5 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/5 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
import xarray as xr
datasets = [ xr.open_dataset(f'../data/raw/{file.name}', engine = 'netcdf4')
                for file in files]
datasets[0]

c:\Users\dmitr\miniconda3\envs\geo\Lib\site-packages\rasterio\__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


['HDF4_EOS:EOS_GRID:../data/raw/MCD12Q1.A2024001.h20v04.061.2025205181956.hdf:MCD12Q1:LC_Type1', 'HDF4_EOS:EOS_GRID:../data/raw/MCD12Q1.A2024001.h20v04.061.2025205181956.hdf:MCD12Q1:LC_Prop2', 'HDF4_EOS:EOS_GRID:../data/raw/MCD12Q1.A2024001.h20v04.061.2025205181956.hdf:MCD12Q1:LC_Prop3', 'HDF4_EOS:EOS_GRID:../data/raw/MCD12Q1.A2024001.h20v04.061.2025205181956.hdf:MCD12Q1:QC', 'HDF4_EOS:EOS_GRID:../data/raw/MCD12Q1.A2024001.h20v04.061.2025205181956.hdf:MCD12Q1:LW', 'HDF4_EOS:EOS_GRID:../data/raw/MCD12Q1.A2024001.h20v04.061.2025205181956.hdf:MCD12Q1:LC_Type2', 'HDF4_EOS:EOS_GRID:../data/raw/MCD12Q1.A2024001.h20v04.061.2025205181956.hdf:MCD12Q1:LC_Type3', 'HDF4_EOS:EOS_GRID:../data/raw/MCD12Q1.A2024001.h20v04.061.2025205181956.hdf:MCD12Q1:LC_Type4', 'HDF4_EOS:EOS_GRID:../data/raw/MCD12Q1.A2024001.h20v04.061.2025205181956.hdf:MCD12Q1:LC_Type5', 'HDF4_EOS:EOS_GRID:../data/raw/MCD12Q1.A2024001.h20v04.061.2025205181956.hdf:MCD12Q1:LC_Prop1_Assessment', 'HDF4_EOS:EOS_GRID:../data/raw/MCD12Q1.A

<xarray.Dataset> Size: 300MB
Dimensions:              (band: 1, x: 2400, y: 2400)
Coordinates:
  * band                 (band) int64 8B 1
  * x                    (x) float64 19kB 2.224e+06 2.225e+06 ... 3.336e+06
  * y                    (y) float64 19kB 5.56e+06 5.559e+06 ... 4.448e+06
    spatial_ref          int64 8B 0
Data variables: (12/13)
    LC_Type1             (band, y, x) float32 23MB ...
    LC_Prop2             (band, y, x) float32 23MB ...
    LC_Prop3             (band, y, x) float32 23MB ...
    QC                   (band, y, x) float32 23MB ...
    LW                   (band, y, x) float32 23MB ...
    LC_Type2             (band, y, x) float32 23MB ...
    ...                   ...
    LC_Type4             (band, y, x) float32 23MB ...
    LC_Type5             (band, y, x) float32 23MB ...
    LC_Prop1_Assessment  (band, y, x) float32 23MB ...
    LC_Prop2_Assessment  (band, y, x) float32 23MB ...
    LC_Prop3_Assessment  (band, y, x) float32 23MB ...
    LC_Prop1             (band, y, x) float32 23MB ...
Attributes: (12/64)
    ALGORITHMPACKAGEACCEPTANCEDATE:     1998-01-01
    ALGORITHMPACKAGEMATURITYCODE:       LAUNCH
    ALGORITHMPACKAGENAME:               MOD12Q1
    ALGORITHMPACKAGEVERSION:            V2.0
    ASSOCIATEDINSTRUMENTSHORTNAME.1:    MODIS
    ASSOCIATEDINSTRUMENTSHORTNAME.2:    MODIS
    ...                                 ...
    SOUTHBOUNDINGCOORDINATE:            40.0
    SPSOPARAMETERS:                     2669
    TileID:                             51020004
    VERSIONID:                          61
    VERTICALTILENUMBER:                 4
    WESTBOUNDINGCOORDINATE:             26.108146

In [ ]:
year_from = 2024

combined = xr.concat(datasets, dim='sounding_id', join='outer')

xco2_ds = combined.where(combined["xco2_quality_flag"] == 0, drop=True)
xco2_ds = xco2_ds.where(xco2_ds["time"].dt.year >= year_from, drop=True)
xco2_ds = xco2_ds.where(
        (xco2_ds["latitude"] >= lat_min)
        & (xco2_ds["latitude"] <= lat_max)
        & (xco2_ds["longitude"] >= lon_min)
        & (xco2_ds["longitude"] <= lon_max),
        drop=True,
    )

xco2_df = xco2_ds[["xco2", "time", "latitude", "longitude"]].to_dataframe().reset_index()
xco2_df = xco2_df.rename(columns={"latitude": "lat", "longitude": "lon"})
xco2_df.head()

,sounding_id,xco2,time,lat,lon
0,2.024010e+15,420.261780,2024-01-01 10:44:00.562007040,44.975578,34.747627
1,2.024010e+15,420.460785,2024-01-01 10:44:00.614993920,44.962967,34.769623
2,2.024010e+15,421.595459,2024-01-01 10:44:00.641990144,44.956535,34.780552
3,2.024010e+15,421.978424,2024-01-01 10:44:00.895016448,44.995373,34.740425
4,2.024010e+15,421.174530,2024-01-01 10:44:00.921007104,44.989140,34.751434
